# Neural network model

The objective of this notebook is to design a neural network model to predict customer churn. Keras library is used for designing and training the neural network.

## Import libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'

## Load data

In [ ]:
path = r'./out/df_clean.csv'

In [ ]:
df = pd.read_csv(path, sep=';', decimal=',')

## EDA

In [ ]:
df.info()

In [ ]:
# Check for null values
df.isnull().sum()

## Data preparation

In [ ]:
# Create new 'prepared' dataframe
df_prep = df.copy()

In [ ]:
# Create dummy-columns for object-columns
df_prep = pd.get_dummies(df_prep, columns=['CLUSTYPE'])
df_prep = pd.get_dummies(df_prep, columns=['CC_CARD'])

# Train and test

In [ ]:
y = df_prep['RESP']
x = df_prep.drop(columns='RESP')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4711)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

## Feature scaling

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Neural network building and model training

In [ ]:
#create an object of sequential model
nn_classifier = Sequential()

# Add the first hidden layer
nn_classifier.add(Dense(units=16, activation='relu', input_dim = 83))

# Adding the second hidden layer
nn_classifier.add(Dense(units=16, activation='relu'))

# Adding the output layer
nn_classifier.add(Dense(units=1, activation='sigmoid'))

In [ ]:
nn_classifier.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Model is trained over 100 epochs
model = nn_classifier.fit(x_train, y_train, validation_split=0.33, batch_size=10, epochs=100)

## Predictions

In [ ]:
# Predictions
y_pred = nn_classifier.predict(x_test)

# Threshold
threshold = 0.7
y_pred_binary = (y_pred > threshold).astype(int)

## Classification report and confusion matrix plotting

In [ ]:
print(classification_report(y_test.astype('int'), y_pred_binary.astype('int')))

In [ ]:
conf_matrix = confusion_matrix(y_test.astype('int'), y_pred_binary.astype('int'))
sn.heatmap(conf_matrix, annot=True,fmt='d', square=True, xticklabels=['not churn', 'churn'], yticklabels=['not churn', 'churn'], linewidths=2, linecolor='w', cmap='Set1')
plt.subplots_adjust(wspace=.3, hspace=.3)

## Visualize Loss and Accuracy

In [ ]:
print(model.history.keys())

In [ ]:
#Accuracy vs Value Accuracy
model.history.keys()

# summarize history for accuracy
plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#loss vs value loss
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prediction and Model Score/Accuracy

In [ ]:
#Model score calculation
score = accuracy_score(y_pred.astype('int'),y_test.astype('int'))
print(score)